In [67]:
import pandas as pd
from my_connector import MySQLConnector

In [69]:
connector = MySQLConnector(host='localhost', user='root', password='root', database='TNT')
connector.connect()

In [31]:
# Define the path to the CSV file
csv_file = 'Datasets/RadaresFijos.csv'

# Define the table name in the database
table_name = 'RadaresFijos'

# Read the CSV file into a pandas DataFrame
df_radar = pd.read_csv(csv_file)
initial_query = f"""
    CREATE TABLE RadaresFijos (
        id INT AUTO_INCREMENT PRIMARY KEY,
        NUM_RADAR INT,
        UBICACION VARCHAR(255),
        CARRETARA_O_VIAL VARCHAR(255),
        UBICACION_CALLE_30 VARCHAR(255),
        PK VARCHAR(255),
        SENTIDO VARCHAR(255),
        TIPO VARCHAR(255),
        LONGITUD FLOAT,
        LATITUD FLOAT,
        COORDENADAS VARCHAR(255)
    )
"""
connector.execute_query(initial_query)

i = 1
for row in df_radar.iterrows():
    distrito_query = f"""
        INSERT INTO RadaresFijos (id, NUM_RADAR, UBICACION, CARRETARA_O_VIAL, UBICACION_CALLE_30, PK, SENTIDO, TIPO, LONGITUD, LATITUD, COORDENADAS) VALUES ('{i}', '{row[1][0]}', '{row[1][1]}', '{row[1][2]}', '{row[1][3]}', '{row[1][4]}', '{row[1][5]}', '{row[1][6]}', '{row[1][7]}', '{row[1][8]}', '{row[1][9]}')
        """
    i = i + 1
    connector.execute_query(distrito_query)

In [50]:
df_luminarias = pd.read_csv('Datasets/iluminacion.csv', low_memory=False)

In [51]:
df_luminarias = df_luminarias.drop(['BARRIO', 'TIPO_BLOQU', 'COD_NDP', 'X_UTM', 'Y_UTM'], axis=1)

In [52]:
df_luminarias['VIA_PAR'] = df_luminarias['VIA_PAR'].fillna('')

In [53]:
df_luminarias['localizacion'] = df_luminarias['VIA_CLASE'] + " " + df_luminarias['VIA_PAR'] + " " + df_luminarias['VIA_NOMBRE'].astype(str)

In [54]:
df_luminarias = df_luminarias.drop(['VIA_CLASE', 'VIA_PAR', 'VIA_CLASE', 'VIA_NOMBRE', 'CLASE_APP', 'NUMERO'], axis=1)

In [55]:
regex = 'NUMERO'
regex2 = 'KILÓMETRO'
df_luminarias['localizacion'] = df_luminarias['localizacion'].str.replace(regex, 'Nº')
df_luminarias['localizacion'] = df_luminarias['localizacion'].str.replace(regex2, 'KM')

In [56]:
df_luminarias['num_luminarias'] = 0

In [57]:
df_luminarias = df_luminarias.groupby(['localizacion', 'DISTRITO', 'TIPO']).count().reset_index()

In [62]:
df_luminarias['TIPO']=df_luminarias['TIPO'].replace({'DESCARGA': 'D', 'LED': 'L', 'LED-DESCARGA': 'E'})

In [72]:
df_luminarias.head()

In [70]:
import uuid
for index, row in df_luminarias.iterrows():
    sql = f"""insert into Luminarias (uuid, localizacion, cod_distrito, tipo, num_luminarias)
            values ('{uuid.uuid4()}', '{row['localizacion']}', {row['DISTRITO']},'{row['TIPO']}', {row['num_luminarias']});"""
    connector.execute_query(sql)

In [73]:
connector.close()